In [1]:
import json
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import numpy as np

# 데이터 가져오기, 학습 데이터셋으로 만들기

## 인벤에서 수집한 데이터 가져오기

In [151]:
immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)
immoral.columns = ['text']
immoral['immoral'] = True

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2151298477.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)


In [152]:
clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)
clean.columns = ['text']
clean['immoral'] = False

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2827705178.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)


## op.gg 데이터 크롤링

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
lii = []
headers = {
    'Referer':'https://www.op.gg/',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
for chm in champ:
    n = 1
    while 1:
        url = f'https://op.gg/api/v1.0/internal/bypass/champions/{chm}/comments?&sort=popular&page={n}&limit=100&hl=ko_KR&is_latest_version=false'
        n += 1
        resp = requests.get(url, headers = headers)
        dict_ch = json.loads(resp.text)
        if len(dict_ch['data']) == 0:
            break
        
        for dat in dict_ch['data']:
            lii.append(dat['content'].replace('\n', ''))
        time.sleep(5)
    
    print(chm)

In [75]:
df_lii = pd.DataFrame(lii, columns = ['text'])
df_lii['immoral'] = ''
df_lii

,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,
1,자기고향 찾아오네,
2,몇달만의 고향이냐,
3,점멸 있을 때 한정 1티어,
4,스킬 사거리 표시 없이 하면 암걸리는 챔,
...,...,...
7968,의외로 꼴짤이 별로 없음,
7969,나피리 근데 초반 라인전도 약하고 카운터도 많음.그리고 돌진기가 다른 암살자와 다르...,
7970,0/6/0 야스오:야이 개쌔꺄 이게 챔이냐 야발4/0/2 나피리:개 맞는데0/8/0...,
7971,얘 한테 지면 사람아닌래끼다버프좀 하고 성능좀 버프히라못 쓸 정도다,


### op.gg 데이터 직접 라벨링, 라벨링 된 csv 가져오기

In [81]:
# df_lii.to_csv(r'data/champions.csv', index = False)
df_lii = pd.read_csv(r'data/champions.csv')

In [195]:
champions = pd.read_csv(r'data\champions.csv')
champions.columns = ['text','immoral']
champions.loc[champions['immoral'] == 0, 'immoral'] = False
champions.loc[champions['immoral'] == 1, 'immoral'] = True
champions = champions.fillna(False)
champions

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2204739627.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  champions.loc[champions['immoral'] == 0, 'immoral'] = False


,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,False
1,자기고향 찾아오네,False
2,몇달만의 고향이냐,False
3,점멸 있을 때 한정 1티어,False
4,스킬 사거리 표시 없이 하면 암걸리는 챔,False
...,...,...
6458,클레드로 굶드라랑 월식 못 가게 너프먹는다니까 진짜 ㅈ같아서 겜 못해먹겠네 월식은 ...,True
6459,진짜 개사기임아니 협곡말고 롤체 학살자 클레드요,False
6460,진짜 태불방 롤백 시킬거 아니면 클레드는 건들면 안되지 ㅋㅋㅋ 선넘네 진짜,False
6461,"""바텀에서 겨우 더블킬을 했다.""""나팔소리가 들렸다.""""여름이었다.""(제압되었습니다!)",False


## Ai hub 데이터 가져오기

In [154]:
# ai hub train set
li = []
for i in range(1, 6):
    with open(r'.\data\train\labled\talksets-train-' + str(i) + '.json') as f:
        data = json.load(f)
    for d in data:
        for s in d['sentences']:
            li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                    'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                    'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
            
train = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])
train['text'] = train['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

train['immoral'] = (train['SEXUAL'] == True) | (train['ABUSE'] == True)
train_set = train[['text', 'immoral']]

In [156]:
# ai hub test set
li = []
with open(r'.\data\valid\labled\talksets-train-6.json') as f:
    data = json.load(f)
for d in data:
    for s in d['sentences']:
        li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
test = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])

test['text'] = test['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

test['immoral'] = (test['SEXUAL'] == True) | (test['ABUSE'] == True)
test_set = test[['text', 'immoral']]

## badwords 가져오기
https://github.com/organization/Gentleman/blob/master/resources/badwords.json

In [184]:
# badwords, train
with open(r".\data\train\labled\badwords.json") as f:
    data = json.load(f)

badwords = pd.DataFrame(data['badwords'], columns= ['text'])
badwords['immoral'] = True

## 가져온 데이터 train/test 나눠서 합치기

In [166]:
im_sl = int(len(immoral) * 0.8)
cl_sl = int(len(clean) * 0.8)
cp_sl = int(len(champions) * 0.8)
bw_sl = int(len(badwords) * 0.8)

In [211]:
train = pd.concat([train_set, immoral[:im_sl], clean[:cl_sl], badwords[:bw_sl], champions[:cp_sl]])
test = pd.concat([test_set, immoral[im_sl:], clean[cl_sl:], badwords[bw_sl:], champions[cp_sl:]])
train['immoral'] = train['immoral'].apply(lambda x : 1 if x == True else 0)
test['immoral'] = test['immoral'].apply(lambda x : 1 if x == True else 0)
train.shape, test.shape

((369610, 2), (46830, 2))

# 데이터 전처리


In [2]:
# train.to_csv(r'data/train_final.csv', index = False)
# test.to_csv(r'data/test_final.csv', index = False)
# train = pd.read_csv(r'data/train_final.csv')
# test = pd.read_csv(r'data/test_final.csv')
train = pd.read_csv(r'data/train_aug.csv')
test = pd.read_csv(r'data/test_aug.csv')

train = train.dropna()
test = test.dropna()
train.shape, test.shape

((441781, 2), (56642, 2))

## 데이터 자모음으로 쪼개기

In [3]:
train['text'] = train['text'].apply(lambda x : j2hcj(h2j(x)))
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))

In [4]:
# swap rows
train = train.sample(frac = 1).reset_index(drop=True)

In [5]:
def textToList(text):
    li = []
    for t in text:
        li.append(list(t.strip()))
    return li

In [6]:
# 자모음 쪼갠 텍스트를 한글자씩 리스트로 만들어서 데이터셋 만든다
def textToList(text):
    li = []
    for t in text:
        li.append(list(t))
    return li

train_list = textToList(train['text'])
train_label = train['immoral']

test_list = textToList(test['text'])
test_label = test['immoral']

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [7]:
# 문장 최대 길이 정해야되니까 문장 길이 분포가 얼마나 되어있는지 확인한다
# li_len = []
# for t in train_list:
#     li_len.append(len(t))
# import seaborn as sns
# sns.set_theme(style="darkgrid")
# sns.histplot(li_len)
## 100으로 자르면 문장 잘리는 아이들 비율이 2%

In [8]:
tokenizer = Tokenizer(num_words=110)
tokenizer.fit_on_texts(train_list) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(train_list) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(test_list)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  166


In [9]:
MAX_SEQUENCE_LENGTH = 100 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of train input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of train label tensor:', y_train.shape) # 감정 데이터 형태 확인
print('Shape of test input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of test label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of train input data tensor: (441781, 100)
Shape of train label tensor: (441781,)
Shape of test input data tensor: (56642, 100)
Shape of test label tensor: (56642,)


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Flatten, Dropout
from tensorflow.keras import layers, models

embedding_dim = 50 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(len(word_vocab)+1, embedding_dim))
model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics=['accuracy',
                                                                            tf.keras.metrics.TrueNegatives(name='true_negatives'),
                                                                            tf.keras.metrics.TruePositives(name='true_positives'),
                                                                            tf.keras.metrics.FalseNegatives(name='false_negatives'),
                                                                            tf.keras.metrics.FalsePositives(name='false_positives')])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          8350      
                                                                 
 conv1d (Conv1D)             (None, None, 256)         38656     
                                                                 
 conv1d_1 (Conv1D)           (None, None, 256)         196864    
                                                                 
 conv1d_2 (Conv1D)           (None, None, 256)         196864    
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                        

In [11]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.2, # new_lr = lr * factor.
    patience=2,
    cooldown=2, # number of epochs to wait before resuming normal operation after lr has been reduced.  
    min_lr=0
)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early, reduce_lr])

model.save('conv_red.h5')

Epoch 1/100
3535/3535 [==============================] - 455s 128ms/step - loss: 0.1408 - accuracy: 0.8112 - true_negatives: 256119.0000 - true_positives: 30595.0000 - false_negatives: 56170.0000 - false_positives: 10540.0000 - val_loss: 0.1163 - val_accuracy: 0.8449 - val_true_negatives: 64631.0000 - val_true_positives: 10024.0000 - val_false_negatives: 11469.0000 - val_false_positives: 2233.0000 - lr: 0.0010
Epoch 2/100
3535/3535 [==============================] - 439s 124ms/step - loss: 0.1132 - accuracy: 0.8517 - true_negatives: 254156.0000 - true_positives: 46871.0000 - false_negatives: 39894.0000 - false_positives: 12503.0000 - val_loss: 0.1093 - val_accuracy: 0.8566 - val_true_negatives: 63663.0000 - val_true_positives: 12023.0000 - val_false_negatives: 9470.0000 - val_false_positives: 3201.0000 - lr: 0.0010
Epoch 3/100
3535/3535 [==============================] - 434s 123ms/step - loss: 0.1074 - accuracy: 0.8604 - true_negatives: 254561.0000 - true_positives: 49527.0000 - false

In [15]:
pred_y_train = model.predict(X_train)
print(model.evaluate(X_train, y_train))

pred_y_test = model.predict(X_test)
print(model.evaluate(X_test, y_test))

13806/13806 [==============================] - 186s 13ms/step - loss: 0.0722 - accuracy: 0.9116 - true_negatives: 321063.0000 - true_positives: 81646.0000 - false_negatives: 26612.0000 - false_positives: 12460.0000
[0.07222520560026169, 0.9115579724311829, 321063.0, 81646.0, 26612.0, 12460.0]
1771/1771 [==============================] - 24s 13ms/step - loss: 0.1066 - accuracy: 0.8673 - true_negatives: 39624.0000 - true_positives: 9503.0000 - false_negatives: 5215.0000 - false_positives: 2300.0000
[0.1066058948636055, 0.8673245906829834, 39624.0, 9503.0, 5215.0, 2300.0]


In [16]:
train_org = pd.read_csv(r'data/train_aug.csv')
train_org = train_org.dropna()
test_org = pd.read_csv(r'data/test_aug.csv')
train_org['pred'] = np.where(pred_y_train > 0.5, 1, 0)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)


In [18]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/96/cf/a714a655266229b51eb2bda117f15275f12457887f165f3c1cc58ab502f1/scikit_learn-1.3.0-cp310-cp310-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.0-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.0-cp310-cp310-win_amd64.whl (9.2 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached threadpoolctl-3.2

In [19]:
from sklearn.metrics import classification_report
print(classification_report(train_org['immoral'], train_org['pred']))
print(classification_report(test_org['immoral'], test_org['pred']))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77    333523
           1       0.24      0.21      0.23    108258

    accuracy                           0.65    441781
   macro avg       0.50      0.50      0.50    441781
weighted avg       0.63      0.65      0.64    441781

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     41924
           1       0.81      0.65      0.72     14718

    accuracy                           0.87     56642
   macro avg       0.84      0.80      0.82     56642
weighted avg       0.86      0.87      0.86     56642



In [ ]:
pred_y_train

array([[0.8909727 ],
       [0.9999909 ],
       [0.03926441],
       ...,
       [0.42057064],
       [1.        ],
       [0.00595657]], dtype=float32)